<a href="https://colab.research.google.com/github/scgil/Colab/blob/main/Recomendador/Recomendador_videojuegos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Recomendación de Videojuegos

El presente Notebook pretende ser un Manual de Uso de la herramienta de Recomendación de videojuegos.

# 1.- Carga de datos

Inicialmente, el Sistema de Recomendación de Videojuegos usará un fichero CSV con 40 mil registros de videojuegos extraígos del Catálogo de Steam. Link al fichero fuente: https://www.kaggle.com/datasets/trolukovich/steam-games-complete-dataset/data

In [ ]:
import pandas as pd

originalData = pd.read_csv('steam_games.csv')
originalData

# 2.- Preprocesamiento con Descripción de Videojuegos



Se realiza un proceso de preprocesamiento de texto utilizando la biblioteca NLTK. El objetivo es tomar un conjunto de datos original, que se asume que contiene al menos una columna de texto, y realizar las siguientes operaciones en sucesión:

Se Tokeniza la Descripción de cada uno de los videojuegos. Se crea un conjunto de Stop Words en inglés (palabras comunes que suelen eliminarse del texto debido a que no aportan información relevante).

Se eliminan las Stop Words de la Descripción Tokenizada y se aplica aplica Stemming (reducir las palabras a su forma raíz) y verificando que sean palabras alfanuméricas.

Finalmente, las palabras preprocesadas se vuelven a unir en una cadena y se agregan a la lista. Se crea un nuevo DataFrame llamado preprocessedData, que es una copia del original, y se añade una nueva columna con el texto preprocesado. El resultado final es un DataFrame con los datos originales y una columna adicional que contiene el texto preprocesado.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in originalData.itertuples():


    text = word_tokenize(row[3]) ## indice de la columna que contiene el texto de Descripción del Videojuego
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedText.append(text)

preprocessedData = originalData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

In [ ]:
preprocessedData.iloc[0]['overview']

In [ ]:
preprocessedData.iloc[0]['processed_text']

# 3.- Creación de la Bolsa de Palabras (Bag of Words) con TF-IDF

Se parte de los datos almacenados en "preprocessedData", en donde para cada Videojuego existe un campo 'preprocessed_text' que contiene la Descripción preprocesada.

El objetivo es transformar todos los textos de las Descripciones de los Videojuegos en vectores de frecuencias (Bag of words), aplicando además la ponderación TF-IDF para los valores de dichas frecuencias.

El paquete sklearn ofrece una clase llamada TfidfVectorizer que crea automáticamente la matriz compuesta por todos los vectores de frecuencias ponderados a partir de un array de textos (preprocessedData['processed_text']).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

In [ ]:
textsBoW.shape

In [ ]:
print(textsBoW)

In [ ]:
bagOfWordsModel.get_feature_names()

In [ ]:
bagOfWordsModel.get_feature_names()[0]

# 4.- Cálculo de distancias entre vectores de frecuencias: Similitud de Coseno

El objetivo final es el de crear una matriz N x N (N= número de Videojuegos) en donde el valor de la posición matriz[i,j] indique la distancia que existe entre el Videojuego i y el Videojuego j.

Gracias a que ahora los textos están representados mediante vectores de frecuencias (textsBoW), el cálculo de distancias se realizará empleando emplearará para ello la Distancia Coseno entre vectores.

In [ ]:
from sklearn.metrics import pairwise_distances

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')

In [ ]:
print(distance_matrix.shape)
print(type(distance_matrix))

# 5.- Búsqueda de los Videojuegos más similares a otro en base a la Descripción

In [ ]:
searchTitle = "DOOM" #Videojuego base para buscar las recomendaciones
indexOfTitle = preprocessedData[preprocessedData['name']==searchTitle].index.values[0]
indexOfTitle

In [ ]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
distance_scores

In [ ]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
ordered_scores

In [ ]:
top_scores = ordered_scores[1:11]
top_scores

In [ ]:
top_indexes = [i[0] for i in top_scores]
top_indexes

In [ ]:
preprocessedData['name'].iloc[top_indexes]